In [ ]:
import pandas as pd
from matplotlib import pyplot
from pandas.plotting import autocorrelation_plot
import numpy as np
from prophet import Prophet

In [ ]:
df = pd.read_csv('entities-2023-09-02_17 33 41.csv', usecols=['start', 'mean'])
df['start'] = pd.to_datetime(df['start'], format='%Y-%m-%d %H:%M:%S')
# df = df.set_index('start')
df = df.tail(8*7*24)
df = df.reset_index(drop=True)
df = df.rename(columns={"start": "ds", "mean": "y"})

train_period = 24*7*4 # Train on prior X weeks
sum_error = 0
forecasts = []
for i in range(4*7):
    m = Prophet()
    period_end = len(df) - (7*4 - i) * 24
    period_start = period_end - train_period
    # print(f"{period_start} - {period_end} / {len(df)}")
    train_data = df[period_start:period_end]
    m.fit(train_data)
    # future = m.make_future_dataframe(periods=24, freq='H')
    pred_df = m.make_future_dataframe(periods=24, freq='H', include_history=False)
    fcst = m.predict(pred_df)
    print(f"{df.loc[period_end]['ds']} {df[period_end:period_end+24]['y'].sum()} {fcst['yhat'].sum()}")
    sum_error += abs(df[period_end:period_end+24]['y'].sum() - fcst['yhat'].sum())
    forecasts.append(fcst)
    # print(f"{df[period_start]['ds']} {df[period_start:period_start + 24]['y'].sum()} {fcst['yhat'].sum()}")
    # fig = m.plot(fcst)
print(sum_error)
forecast = pd.concat(forecasts)

pyplot.figure()
fig, ax = pyplot.subplots(figsize=(15, 5))
start = 4*7*24
df[start:].plot(x='ds', y='y', ax=ax)
forecast.plot(x='ds', y='yhat', ax=ax)
ax.fill_between(forecast['ds'].dt.to_pydatetime(), forecast['yhat_lower'], forecast['yhat_upper'], color='k', alpha=0.1);    
pyplot.legend()
pyplot.show()
# (df['mean'][start:] - df['pred'][start:]).rolling(12).mean().plot(ax=ax)